In [1]:
trainingjobName = 'qoe_test'    # Replace with the training job name you created
epochs = 1
version = 1

In [2]:
import tensorflow as tf
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Dropout, Activation
from tensorflow.keras.layers import LSTM
import numpy as np
print("numpy version")
print(np.__version__)
import pandas as pd
import os
from featurestoresdk.feature_store_sdk import FeatureStoreSdk
from modelmetricsdk.model_metrics_sdk import ModelMetricsSdk

fs_sdk = FeatureStoreSdk()
mm_sdk = ModelMetricsSdk()
print("job name is: ", trainingjobName)
features = fs_sdk.get_features(trainingjobName, ['pdcpBytesDl','pdcpBytesUl'])
print("Dataframe:")
print(features)

features_cellc2b2 = features
print(features_cellc2b2)
print('Previous Data Types are --> ', features_cellc2b2.dtypes)
features_cellc2b2["pdcpBytesDl"] = pd.to_numeric(features_cellc2b2["pdcpBytesDl"], downcast="float")
features_cellc2b2["pdcpBytesUl"] = pd.to_numeric(features_cellc2b2["pdcpBytesUl"], downcast="float")
print('New Data Types are --> ', features_cellc2b2.dtypes)

features_cellc2b2 = features_cellc2b2[['pdcpBytesDl', 'pdcpBytesUl']]

def split_series(series, n_past, n_future):
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)
X, y = split_series(features_cellc2b2.values,10, 1)
X = X.reshape((X.shape[0], X.shape[1],X.shape[2]))
y = y.reshape((y.shape[0], y.shape[2]))
print(X.shape)
print(y.shape)

model = Sequential()
model.add(LSTM(units = 150, activation="tanh" ,return_sequences = True, input_shape = (X.shape[1], X.shape[2])))

model.add(LSTM(units = 150, return_sequences = True,activation="tanh"))

model.add(LSTM(units = 150,return_sequences = False,activation="tanh" ))

model.add((Dense(units = X.shape[2])))

model.compile(loss='mse', optimizer='adam',metrics=['mse'])
model.summary()

model.fit(X, y, batch_size=10,epochs=int(epochs), validation_split=0.2)
yhat = model.predict(X, verbose = 0)


xx = y
yy = yhat
model.save("./output")
import json
data = {}
data['metrics'] = []
data['metrics'].append({'Accuracy': str(np.mean(np.absolute(np.asarray(xx)-np.asarray(yy))<5))})

2024-08-29 13:29:30.719165: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-29 13:29:30.719292: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


numpy version
1.24.4
job name is:  qoe_test
2024-08-29 13:29:33,182 | feature_store_sdk.py 85 get_features() |  DEBUG | "pdcpBytesDl" does not have empty space
2024-08-29 13:29:33,183 | feature_store_sdk.py 85 get_features() |  DEBUG | "pdcpBytesUl" does not have empty space
2024-08-29 13:29:33,184 | feature_store_sdk.py 127 build_fetch_query() |  DEBUG | Check Select query--> select "pdcpBytesDl", "pdcpBytesUl" from qoe_test ;
2024-08-29 13:29:33,414 | feature_store_sdk.py 93 get_features() |  DEBUG | Non spaced pd  pdcpBytesDl pdcpBytesUl
0         0.0         0.0
1         0.0         0.0
2         0.0         0.0
3         0.0         0.0
4         0.0         0.0
2024-08-29 13:29:33,417 | feature_store_sdk.py 108 get_features() |  DEBUG | Select pipeline merged_df --> 
  pdcpBytesDl pdcpBytesUl
0         0.0         0.0
1         0.0         0.0
2         0.0         0.0
3         0.0         0.0
4         0.0         0.0
Dataframe:
      pdcpBytesDl pdcpBytesUl
0             0.0 

2024-08-29 13:29:33.493324: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-08-29 13:29:33.493350: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-08-29 13:29:33.493364: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debug-pod): /proc/driver/nvidia/version does not exist
2024-08-29 13:29:33.495050: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 150)           91800     
                                                                 
 lstm_1 (LSTM)               (None, 10, 150)           180600    
                                                                 
 lstm_2 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 2)                 302       
                                                                 
Total params: 453,302
Trainable params: 453,302
Non-trainable params: 0
_________________________________________________________________
3241/3241 [==============================] - 72s 22ms/step - loss: 768909.6250 - mse: 768909.6250 - val_loss: 745143.5000 - val_mse: 745143.5000


2024-08-29 13:31:01.933663: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./assets


INFO:tensorflow:Assets written to: ./assets
